In [1]:
import numpy as np

from qiskit import QuantumCircuit, pulse, circuit, transpile, assemble, schedule
from qiskit.extensions import HamiltonianGate
from qiskit.pulse.library import GaussianSquare
from qiskit.providers.fake_provider import FakeAthens, FakeBoeblingen, FakeOpenPulse2Q, FakeBoeblingenV2, FakeAthensV2, FakeBelem, FakeBogota, FakeOpenPulse3Q
# from hal_params import init_static_ham

Here is all information about the fake backend in this case:

In [2]:
from qiskit.providers.fake_provider import FakeJakarta
# backend = FakeBoeblingen()
backend = FakeAthens()

# props = backend.meas_map 
# props
dt = backend.configuration().dt

backend.configuration().to_dict()

{'backend_name': 'fake_athens',
 'backend_version': '1.3.13',
 'n_qubits': 5,
 'basis_gates': ['id', 'rz', 'sx', 'x', 'cx', 'reset'],
 'gates': [{'name': 'id',
   'parameters': [],
   'qasm_def': 'gate id q { U(0, 0, 0) q; }',
   'coupling_map': [[0], [1], [2], [3], [4]]},
  {'name': 'rz',
   'parameters': ['theta'],
   'qasm_def': 'gate rz(theta) q { U(0, 0, theta) q; }',
   'coupling_map': [[0], [1], [2], [3], [4]]},
  {'name': 'sx',
   'parameters': [],
   'qasm_def': 'gate sx q { U(pi/2, 3*pi/2, pi/2) q; }',
   'coupling_map': [[0], [1], [2], [3], [4]]},
  {'name': 'x',
   'parameters': [],
   'qasm_def': 'gate x q { U(pi, 0, pi) q; }',
   'coupling_map': [[0], [1], [2], [3], [4]]},
  {'name': 'cx',
   'parameters': [],
   'qasm_def': 'gate cx q0, q1 { CX q0, q1; }',
   'coupling_map': [[0, 1],
    [1, 0],
    [1, 2],
    [2, 1],
    [2, 3],
    [3, 2],
    [3, 4],
    [4, 3]]},
  {'name': 'reset', 'parameters': None, 'qasm_def': None}],
 'local': False,
 'simulator': False,
 'cond

In [3]:
def describe_qubit(qubit, properties):
    """Print a string describing some of reported properties of the given qubit."""

    # Conversion factors from standard SI units
    us = 1e6
    ns = 1e9
    GHz = 1e-9

    print("Qubit {0} has a \n"
          "  - T1 time of {1} mus\n"
          "  - T2 time of {2} mus\n"
          "  - U2 gate error of {3}\n"
          "  - U2 gate duration of {4} nanoseconds\n"
          "  - resonant frequency of {5} GHz".format(
              qubit,
              properties.t1(qubit) * us,
              properties.t2(qubit) * us,
              properties.gate_error('sx', qubit),
              properties.gate_length('sx', qubit) * ns,
              properties.frequency(qubit) * GHz))

props = backend.properties()
print(describe_qubit(0, props))
print(describe_qubit(1, props))

Qubit 0 has a 
  - T1 time of 63.48783021708388 mus
  - T2 time of 112.23245535599808 mus
  - U2 gate error of 0.00037753800551696055
  - U2 gate duration of 35.55555555555556 nanoseconds
  - resonant frequency of 5.1753836395136075 GHz
None
Qubit 1 has a 
  - T1 time of 73.093518301544 mus
  - T2 time of 126.83382141649017 mus
  - U2 gate error of 0.00030732363305963013
  - U2 gate duration of 35.55555555555556 nanoseconds
  - resonant frequency of 5.267216864382969 GHz
None


In [4]:
backend.properties().to_dict()

{'backend_name': 'ibmq_athens',
 'backend_version': '1.3.13',
 'last_update_date': datetime.datetime(2021, 3, 15, 14, 10, 39, tzinfo=tzoffset(None, -14400)),
 'qubits': [[{'date': datetime.datetime(2021, 3, 15, 0, 15, 3, tzinfo=tzoffset(None, -14400)),
    'name': 'T1',
    'unit': 'us',
    'value': 63.487830217083875},
   {'date': datetime.datetime(2021, 3, 15, 0, 16, 8, tzinfo=tzoffset(None, -14400)),
    'name': 'T2',
    'unit': 'us',
    'value': 112.23245535599808},
   {'date': datetime.datetime(2021, 3, 15, 14, 10, 39, tzinfo=tzoffset(None, -14400)),
    'name': 'frequency',
    'unit': 'GHz',
    'value': 5.1753836395136075},
   {'date': datetime.datetime(2021, 3, 15, 14, 10, 39, tzinfo=tzoffset(None, -14400)),
    'name': 'anharmonicity',
    'unit': 'GHz',
    'value': -0.33610237055834047},
   {'date': datetime.datetime(2021, 3, 15, 0, 14, 2, tzinfo=tzoffset(None, -14400)),
    'name': 'readout_error',
    'unit': '',
    'value': 0.010099999999999998},
   {'date': datetime

Model of the Hamiltonian:

In [5]:
from IPython import display
from IPython.core.display import HTML
backend.configuration().hamiltonian["h_latex"]

'\\begin{align} \\mathcal{H}/\\hbar = & \\sum_{i=0}^{4}\\left(\\frac{\\omega_{q,i}}{2}(\\mathbb{I}-\\sigma_i^{z})+\\frac{\\Delta_{i}}{2}(O_i^2-O_i)+\\Omega_{d,i}D_i(t)\\sigma_i^{X}\\right) \\\\ & + J_{1,2}(\\sigma_{1}^{+}\\sigma_{2}^{-}+\\sigma_{1}^{-}\\sigma_{2}^{+}) + J_{3,4}(\\sigma_{3}^{+}\\sigma_{4}^{-}+\\sigma_{3}^{-}\\sigma_{4}^{+}) + J_{0,1}(\\sigma_{0}^{+}\\sigma_{1}^{-}+\\sigma_{0}^{-}\\sigma_{1}^{+}) + J_{2,3}(\\sigma_{2}^{+}\\sigma_{3}^{-}+\\sigma_{2}^{-}\\sigma_{3}^{+}) \\\\ & + \\Omega_{d,0}(U_{0}^{(0,1)}(t))\\sigma_{0}^{X} + \\Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{2}^{(1,2)}(t))\\sigma_{1}^{X} \\\\ & + \\Omega_{d,2}(U_{3}^{(2,1)}(t)+U_{4}^{(2,3)}(t))\\sigma_{2}^{X} + \\Omega_{d,3}(U_{6}^{(3,4)}(t)+U_{5}^{(3,2)}(t))\\sigma_{3}^{X} \\\\ & + \\Omega_{d,4}(U_{7}^{(4,3)}(t))\\sigma_{4}^{X} \\\\ \\end{align}'

\begin{align} 
    \mathcal{H}/\hbar = & \sum_{i=0}^{4}\left(\frac{\omega_{q,i}}{2}(\mathbb{I}-\sigma_i^{z})+\frac{\Delta_{i}}{2}(O_i^2-O_i)+\Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) + J_{3,4}(\sigma_{3}^{+}\sigma_{4}^{-}+\sigma_{3}^{-}\sigma_{4}^{+}) + J_{0,1}(\sigma_{0}^{+}\sigma_{1}^{-}+\sigma_{0}^{-}\sigma_{1}^{+}) + J_{2,3}(\sigma_{2}^{+}\sigma_{3}^{-}+\sigma_{2}^{-}\sigma_{3}^{+}) \\ & + \Omega_{d,0}(U_{0}^{(0,1)}(t))\sigma_{0}^{X} + \Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{2}^{(1,2)}(t))\sigma_{1}^{X} \\ & + \Omega_{d,2}(U_{3}^{(2,1)}(t)+U_{4}^{(2,3)}(t))\sigma_{2}^{X} + \Omega_{d,3}(U_{6}^{(3,4)}(t)+U_{5}^{(3,2)}(t))\sigma_{3}^{X} \\ & + \Omega_{d,4}(U_{7}^{(4,3)}(t))\sigma_{4}^{X} \\ 
\end{align}

In [6]:
print(HTML(backend.configuration().hamiltonian["description"]))

<IPython.core.display.HTML object>
